In [ ]:
## Import modules
import pandas as pd
import numpy as np

import datetime
from   datetime import datetime, date
# pd.set_option('display.notebook_repr_html', False)
# pd.set_option('display.max_columns', 7)
# pd.set_option('display.max_rows', 10)
# pd.set_option('display.width', 60)

import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

import os
from os import listdir
from os.path import isfile, join
import glob
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

In [ ]:
## Using merged files

os.chdir('최종합본')


files_for_day = []
for file in glob.glob("*.csv"):
    files_for_day.append(file)

def minus_to_zero(x):
    if x < 0:
        return 0
    else:
        return x

## Make dataset for train
for i in range(len(files_for_day)):
    tmp_data = pd.read_csv(files_for_day[i])
    tmp_data['Time_conv'] = pd.to_datetime(tmp_data['Time_conv'], format = "%Y/%m/%d %H:%M:%S.%f")
    tmp_data = tmp_data[540:-900]
    tmp_data['Time_conv_day'] = pd.to_datetime(tmp_data['Time_conv'].map(lambda x: x.strftime('%Y/%m/%d')))
    tmp_data[files_for_day[i][:-4]+'_전류값1'] = tmp_data[files_for_day[i][:-4]+'_전류값1'].apply(minus_to_zero)
    grouped_data_feature = tmp_data.groupby('Time_conv_day', as_index=False).mean()
    grouped_data_target = tmp_data.groupby('Time_conv_day', as_index=False).sum()
    tmp_data_fin = grouped_data_feature
    tmp_data_fin[[files_for_day[i][:-4]+'_유량1', files_for_day[i][:-4]+'_전류값1']] = grouped_data_target[[files_for_day[i][:-4]+'_유량1', files_for_day[i][:-4]+'_전류값1']]
    tmp_data_fin[[files_for_day[i][:-4]+'_유량1', files_for_day[i][:-4]+'_전류값1']] = tmp_data_fin[[files_for_day[i][:-4]+'_유량1', files_for_day[i][:-4]+'_전류값1']].shift(periods=-1)
    tmp_data_fin = tmp_data_fin[:-1]
    tmp_data_fin.to_csv('day_'+files_for_day[i], index=False)

In [ ]:
## Setting models

def LR_result(X_train, y_train, X_test, y_test):
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_preds = lr.predict(X_test)
    y_preds2 = lr.predict(X_train)
    mse = mean_squared_error(y_test, y_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_preds)
    mse2 = mean_squared_error(y_train, y_preds2)
    rmse2 = np.sqrt(mse2)
    mae2 = mean_absolute_error(y_train, y_preds2)
    result_test = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse, rmse, mae)
    result_train = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse2, rmse2, mae2)
    return {'Test score' : result_test, 'Test Prediction' :y_preds, 'Test True':np.array(y_test),
            'Train score' : result_train, 'Train Prediction' :y_preds2, 'Train True':np.array(y_train)}

def LAS_result(X_train, y_train, X_test, y_test):
    las = Lasso()
    las.fit(X_train, y_train)
    y_preds = las.predict(X_test)
    y_preds2 = las.predict(X_train)
    mse = mean_squared_error(y_test, y_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_preds)
    mse2 = mean_squared_error(y_train, y_preds2)
    rmse2 = np.sqrt(mse2)
    mae2 = mean_absolute_error(y_train, y_preds2)
    result_test = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse, rmse, mae)
    result_train = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse2, rmse2, mae2)
    return {'Test score' : result_test, 'Test Prediction' :y_preds, 'Test True':np.array(y_test),
            'Train score' : result_train, 'Train Prediction' :y_preds2, 'Train True':np.array(y_train)}

def RID_result(X_train, y_train, X_test, y_test):
    rid = Ridge()
    rid.fit(X_train, y_train)
    y_preds = rid.predict(X_test)
    y_preds2 = rid.predict(X_train)
    mse = mean_squared_error(y_test, y_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_preds)
    mse2 = mean_squared_error(y_train, y_preds2)
    rmse2 = np.sqrt(mse2)
    mae2 = mean_absolute_error(y_train, y_preds2)
    result_test = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse, rmse, mae)
    result_train = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse2, rmse2, mae2)
    return {'Test score' : result_test, 'Test Prediction' :y_preds, 'Test True':np.array(y_test),
            'Train score' : result_train, 'Train Prediction' :y_preds2, 'Train True':np.array(y_train)}

def ELA_result(X_train, y_train, X_test, y_test):
    ela = ElasticNet()
    ela.fit(X_train, y_train)
    y_preds = ela.predict(X_test)
    y_preds2 = ela.predict(X_train)
    mse = mean_squared_error(y_test, y_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_preds)
    mse2 = mean_squared_error(y_train, y_preds2)
    rmse2 = np.sqrt(mse2)
    mae2 = mean_absolute_error(y_train, y_preds2)
    result_test = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse, rmse, mae)
    result_train = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse2, rmse2, mae2)
    return {'Test score' : result_test, 'Test Prediction' :y_preds, 'Test True':np.array(y_test),
            'Train score' : result_train, 'Train Prediction' :y_preds2, 'Train True':np.array(y_train)}

def SVR_result(X_train, y_train, X_test, y_test):
    svr = SVR()
    svr.fit(X_train, y_train)
    y_preds = svr.predict(X_test)
    y_preds2 = svr.predict(X_train)
    mse = mean_squared_error(y_test, y_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_preds)
    mse2 = mean_squared_error(y_train, y_preds2)
    rmse2 = np.sqrt(mse2)
    mae2 = mean_absolute_error(y_train, y_preds2)
    result_test = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse, rmse, mae)
    result_train = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse2, rmse2, mae2)
    return {'Test score' : result_test, 'Test Prediction' :y_preds, 'Test True':np.array(y_test),
            'Train score' : result_train, 'Train Prediction' :y_preds2, 'Train True':np.array(y_train)}

def RFR_result(X_train, y_train, X_test, y_test):
    rfr = RandomForestRegressor()
    rfr.fit(X_train, y_train)
    y_preds = rfr.predict(X_test)
    y_preds2 = rfr.predict(X_train)
    mse = mean_squared_error(y_test, y_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_preds)
    mse2 = mean_squared_error(y_train, y_preds2)
    rmse2 = np.sqrt(mse2)
    mae2 = mean_absolute_error(y_train, y_preds2)
    result_test = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse, rmse, mae)
    result_train = 'MSE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:.3f}'.format(mse2, rmse2, mae2)
    return {'Test score' : result_test, 'Test Prediction' :y_preds, 'Test True':np.array(y_test),
            'Train score' : result_train, 'Train Prediction' :y_preds2, 'Train True':np.array(y_train)}

In [ ]:
## Cross-Validation

os.chdir('H:\gimys\최종합본\day1007')
files_for_day = []
for file in glob.glob("*.csv"):
    files_for_day.append(file)

def model_result_dict(files_for_day):
    result_dict = {}
    # for j in range(len(files_for_day)):
    for j in range(len(files_for_day)):
        tmp_data_fin = pd.read_csv(files_for_day[j])
        tmp_data_fin['Target'] = tmp_data_fin[files_for_day[j][4:-4]+'_전류값1']/tmp_data_fin[files_for_day[j][4:-4]+'_유량1']
        tmp_data_fin['Day'] = tmp_data_fin['Time_conv_day'].astype(str).str[8:10]
        
        tmp_data_fin = tmp_data_fin.replace(np.NaN, 0)
        tmp_data_fin = tmp_data_fin.replace(np.inf, 0)
        tmp_data_fin = tmp_data_fin.replace(-(np.inf), 0)
        tmp_data_fin_modi = tmp_data_fin.iloc[np.where(tmp_data_fin['Target'] != 0)[0]]
        tmp_data_fin_modi = tmp_data_fin_modi.iloc[np.where((tmp_data_fin_modi['Target'] <= 0.4) & (tmp_data_fin_modi['Target'] >= 0.1))[0]]
        
        ## index split
        X = tmp_data_fin_modi.values
        set_splits = 5
        splits = TimeSeriesSplit(n_splits=set_splits)
    
        ## set feature and target
        index_list = tmp_data_fin_modi.columns.tolist()
        index_list.pop(index_list.index(files_for_day[j][4:-4]+'_유량1'))
        index_list.pop(index_list.index(files_for_day[j][4:-4]+'_전류값1'))
        index_list.pop(index_list.index('Target'))
        index_list.pop(index_list.index('Time_conv_day'))
        feature_label = index_list
        target_label = ['Target']
        
        
#         for k in range(set_splits):
#             tmp_train = tmp_data_fin_modi.iloc[list(splits.split(X))[k][0]]
#             tmp_test = tmp_data_fin_modi.iloc[list(splits.split(X))[k][1]]
#             X_train, y_train = tmp_train[feature_label], tmp_train[target_label]
#             X_test, y_test = tmp_test[feature_label], tmp_test[target_label]
#             result_dict[files_for_day[j][:-4]+'_sp'+str(k)] =  LR_result(X_train, y_train, X_test, y_test)
        k = 4
        tmp_train = tmp_data_fin_modi.iloc[list(splits.split(X))[k][0]]
        tmp_test = tmp_data_fin_modi.iloc[list(splits.split(X))[k][1]]
        X_train, y_train = tmp_train[feature_label], tmp_train[target_label]
        X_test, y_test = tmp_test[feature_label], tmp_test[target_label]
        result_dict[files_for_day[j][:-4]+'_sp'+str(k)] =  RFR_result(X_train, y_train, X_test, y_test)
    return result_dict


## modify dataset
y_train_modi = y_train.iloc[np.where(y_train != 0)[0]]
X_train_modi = X_train.iloc[np.where(y_train != 0)[0]]
y_test_modi = y_test.iloc[np.where(y_test != 0)[0]]
X_test_modi = X_test.iloc[np.where(y_test != 0)[0]]

In [ ]:
## Make 'D-1 ~ D-5' data (Data Shift)
### add weather data
data = pd.read_csv('H:\gimys\weather_data.csv', encoding='euc-kr')

for i in (range(1, 6)):
    globals()["data_d_" + str(i)] = data.shift(i)

naming = list()
for j in range(1,6):
    for i in range(len(list(data.columns))):        
        naming.append(list(globals()["data_d_" + str(j)].columns)[i]+'_d-{}'.format(j))
    globals()["data_d_" + str(j)].set_axis(naming, axis = 1)
    naming = list()
total_data = pd.concat([data, data_d_1, data_d_2, data_d_3, data_d_4, data_d_5], axis = 1)

total_data.to_csv('weather_d5.csv', index=False)
del total_data

weather_data = pd.read_csv('weather_d5.csv')
weather_data =weather_data.drop(columns=['일시', '일시_d-5', '일시_d-4', '일시_d-3', '일시_d-2', '일시_d-1'])


## Modeling using only shifted data

os.chdir('H:\gimys\최종합본\day1007')
files_for_day = []
for file in glob.glob("*.csv"):
    files_for_day.append(file)
# weather_data = pd.read_csv('../../weather_data.csv', encoding = 'euc-kr')
# weather_data = weather_data.drop(columns=['지점', 'Month', 'Year', 'Weekday', 'Time'])
# weather_data = weather_data.rename(columns = {'일시' : 'Time_conv_day'}).copy()
for k in range(len(files_for_day)):
    data = pd.read_csv(files_for_day[k])
#     data = data.drop(columns=['Time_conv_day'])
    
    for m in (range(1, 6)):
        globals()["data_d_" + str(m)] = data.shift(m)
    
    naming = list()
    for j in range(1,6):
        for i in range(len(list(data.columns))):        
            naming.append(list(globals()["data_d_" + str(j)].columns)[i]+'_d-{}'.format(j))
        globals()["data_d_" + str(j)].set_axis(naming, axis = 1)
        naming = list()
    total_data = pd.concat([data, data_d_1, data_d_2, data_d_3, data_d_4, data_d_5], axis = 1)
    total_data = total_data[5:]
#     total_weather_data = pd.merge(total_data, weather_data, how='left', on = 'Time_conv_day')
    total_data =total_data.drop(columns=['Time_conv_day', 'Time_conv_day_d-5', 'Time_conv_day_d-4', 'Time_conv_day_d-3',
                                                         'Time_conv_day_d-2', 'Time_conv_day_d-1'])    
    total_data.to_csv('shift'+files_for_day[k], index=False)
    print(files_for_day[k])
    
## Apply Data Shift
def model_result_dict_shift(files_for_day):
    result_dict = {}
    # for j in range(len(files_for_day)):
    for j in range(len(files_for_day)):
        tmp_data_fin = pd.read_csv(files_for_day[j])
        tmp_data_fin = pd.get_dummies(tmp_data_fin)
        tmp_data_fin['Target'] = tmp_data_fin[files_for_day[j][9:-4]+'_전류값1']/tmp_data_fin[files_for_day[j][9:-4]+'_유량1']
        tmp_data_fin['Target_d-1'] = tmp_data_fin[files_for_day[j][9:-4]+'_전류값1_d-1']/tmp_data_fin[files_for_day[j][9:-4]+'_유량1_d-1']
        tmp_data_fin['Target_d-2'] = tmp_data_fin[files_for_day[j][9:-4]+'_전류값1_d-2']/tmp_data_fin[files_for_day[j][9:-4]+'_유량1_d-2']
        tmp_data_fin['Target_d-3'] = tmp_data_fin[files_for_day[j][9:-4]+'_전류값1_d-3']/tmp_data_fin[files_for_day[j][9:-4]+'_유량1_d-3']
        tmp_data_fin['Target_d-4'] = tmp_data_fin[files_for_day[j][9:-4]+'_전류값1_d-4']/tmp_data_fin[files_for_day[j][9:-4]+'_유량1_d-4']
        tmp_data_fin['Target_d-5'] = tmp_data_fin[files_for_day[j][9:-4]+'_전류값1_d-5']/tmp_data_fin[files_for_day[j][9:-4]+'_유량1_d-5']
        

        tmp_data_fin = tmp_data_fin.replace(np.NaN, 0)
        tmp_data_fin = tmp_data_fin.replace(np.inf, 0)
        tmp_data_fin = tmp_data_fin.replace(-(np.inf), 0)
        tmp_data_fin_modi = tmp_data_fin.iloc[np.where(tmp_data_fin['Target'] != 0)[0]]
        tmp_data_fin_modi = tmp_data_fin_modi.iloc[np.where((tmp_data_fin_modi['Target'] <= 0.4) & (tmp_data_fin_modi['Target'] >= 0.1))[0]]        
        ## index split
        X = tmp_data_fin_modi.values
        set_splits = 5
        splits = TimeSeriesSplit(n_splits=set_splits)
    
        ## set feature and target
        ## 유량과 전류값은 target에 포함되므로 제거하고, Time_conv_day는 분리된 Year, Month, Day가 대체
        index_list = tmp_data_fin_modi.columns.tolist()
        index_list.pop(index_list.index(files_for_day[j][9:-4]+'_유량1'))
        index_list.pop(index_list.index(files_for_day[j][9:-4]+'_전류값1'))
        index_list.pop(index_list.index('Target'))
        feature_label = index_list
        target_label = ['Target']
        
        
#         for k in range(set_splits):
#             tmp_train = tmp_data_fin_modi.iloc[list(splits.split(X))[k][0]]
#             tmp_test = tmp_data_fin_modi.iloc[list(splits.split(X))[k][1]]
#             X_train, y_train = tmp_train[feature_label], tmp_train[target_label]
#             X_test, y_test = tmp_test[feature_label], tmp_test[target_label]
#             result_dict[files_for_day[j][:-4]+'_sp'+str(k)] =  LR_result(X_train, y_train, X_test, y_test)
        k = 4
        tmp_train = tmp_data_fin_modi.iloc[list(splits.split(X))[k][0]]
        tmp_test = tmp_data_fin_modi.iloc[list(splits.split(X))[k][1]]
        X_train, y_train = tmp_train[feature_label], tmp_train[target_label]
        X_test, y_test = tmp_test[feature_label], tmp_test[target_label]
        result_dict[files_for_day[j][:-4]+'_sp'+str(k)] =  RFR_result(X_train, y_train, X_test, y_test)            
    return result_dict


save_result_shift = model_result_dict_shift(files_for_day)